In [1]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

In [2]:
# Database Setup
engine = create_engine("sqlite:///db/gunviolence_db.sqlite")
df_gun_violence = pd.read_sql('select * from gunviolence_db', engine)
print(len(df_gun_violence))

425


In [3]:
@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")

In [4]:
###### QUESTION: why need to change the variable list_states -> list_states2?

@app.route("/states")
def list_states():
    """Return a list of states."""

    # Use Pandas to perform the sql query
    engine = create_engine("sqlite:///db/gunviolence_db.sqlite")
    df_gun_violence = pd.read_sql('select * from gunviolence_db', engine)
    
    stmt = list(df_gun_violence['State'].unique())

    # Return a list of the column names (state names)
    return jsonify(stmt)

print(stmt)
print(list_states)

NameError: name 'stmt' is not defined

In [ ]:
# METADATA TABLE: Create a summary of gun violence by state. 
@app.route("/metadata/<state>")
def gunviolence_metadata(state):
    """Return the gun violence data summary (metadata) for a given state."""

    engine = create_engine("sqlite:///db/gunviolence_db.sqlite")
    df_gun_violence2 = pd.read_sql(f"select * from gunviolence_db where State = '{state}' ", engine)
    results = [
        list(df_gun_violence2['State']),
        list(df_gun_violence2['year']),
        list(df_gun_violence2['Incident ID']),
        list(df_gun_violence2['Killed'],
        list(df_gun_violence2['Injured'])
    ]
    print('results:', results)
            
    # Create a dictionary entry for each row of gun violence summary data (metadata)
    metadata = 
             
    # Create a dictionary entry for each row of gun violence summary data (metadata)
  #  gunviolence_metadata = {}
   # for result in results:
    #    print(result)
        # 
     #   gunviolence_metadata["year"] = result[0]
      #  gunviolence_metadata["Incident_ID"] = result[1]
        # gunviolence_metadata["Number_Killed"].sum = result[2]
        # gunviolence_metadata["Number_Injured"].sum = result[3]

    # print(gunviolence_metadata)
    #return jsonify(gunviolence_metadata)

>>> df = pd.DataFrame([['a', 'b'], ['c', 'd']],
...                   index=['row 1', 'row 2'],
...                   columns=['col 1', 'col 2'])
>>> df.to_json(orient='split')
'{"columns":["col 1","col 2"],
  "index":["row 1","row 2"],
  "data":[["a","b"],["c","d"]]}'

In [ ]:
# GOAL: Create a dictionary entry for each row of gun violence summary data (metadata)
# { 'year' : 2018
#   'incident_count': "IncidentID".count,
#   'number_killed': "Killed".sum,
#    'number_injured': "Injured".sum
# }